In [ ]:
from pathlib import Path
import datetime

import matplotlib.pyplot as plt
from matplotlib import gridspec
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import stock

In [ ]:
# 新高値銘柄のバックテスト
code_df = stock.get_code_df()
code_list = code_df["コード"].to_list()

target_date = datetime.date(year=2023, month=6, day=1)
highest_codes = []
for code in code_list:
    df = stock.kabutan.read_data_csv(stock.DATA_DIR / "daily" / f"{code}.csv")
    df = df.filter((pl.col("date") <= target_date) & (pl.col("date") >= target_date - datetime.timedelta(days=365)))
    if len(df) == 0:
        continue
    if df["high"].max() * 0.998 < df["high"][-1]:
        highest_codes.append(code)

In [ ]:
len(highest_codes)

In [ ]:
idx = 0
code = highest_codes[idx]
df = stock.kabutan.read_data_csv(stock.DATA_DIR / "daily" / f"{code}.csv")
df = df.filter((pl.col("date") <= target_date) & (pl.col("date") >= target_date - datetime.timedelta(days=365 * 2)))
# Create subplots and mention plot grid size
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'), row_width=[0.2, 0.7])
# Plot OHLC on 1st row
fig.add_trace(go.Candlestick(x=df["date"], open=df["open"], high=df["high"], low=df["low"], close=df["close"], name="OHLC"),  row=1, col=1)
# Bar trace for volumes on 2nd row without legend
fig.add_trace(go.Bar(x=df['date'], y=df['volume'], showlegend=False), row=2, col=1)
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
fig.update_layout(width=1500, height=1000)
fig.show()

In [ ]:
# 値動きと出来高から候補を抽出

gs = gridspec.GridSpec(3, 2)
fig = plt.figure(figsize=(40, 8))
ax1 = plt.subplot(gs[:2, 0])
ax2 = plt.subplot(gs[2, 0], sharex=ax1)
ax1.plot(df["date"], df["close"])
ax2.bar(df["date"], df["volume"])
ax2.set_ylim(0, df["volume"].max())
ax1.set_yscale("log")
ax1.grid()
ax2.grid()

recent_df = df.filter(pl.col("date") >= target_date - datetime.timedelta(120))
ax3 = plt.subplot(gs[:2, 1])
ax4 = plt.subplot(gs[2:, 1], sharex=ax3)
ax3.plot(recent_df["date"], recent_df["close"])
ax4.bar(recent_df["date"], recent_df["volume"])
ax4.set_ylim(0, df["volume"].mean() * 2)
ax3.set_yscale("log")
ax3.grid()
ax4.grid()

fig.show()